# 과제 (100점)

#### 본인이 관심 있는 키워드에 대해서 해당 크롤링을 반복해서 수행하시오. 데이터를 수집하여 excel 파일로 저장하시오.

In [1]:
#1. 필요한 라이브러리 import 하기 (10점)
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import pandas as pd
import re

In [2]:
#2. 본인이 관심 있는 키워드에 대해서 크롤링을 수행하고 상품명, 가격, 별점 정보 저장하기 (40점)
driver = webdriver.Chrome()
driver.get("https://www.gmarket.co.kr")

search_box = driver.find_element(By.NAME, 'keyword')
search_box.send_keys('축구공')
search_box.send_keys(Keys.RETURN)

driver.implicitly_wait(3)

search_results = driver.find_elements(By.CSS_SELECTOR, 'div.box__information') 

title_list = list()
price_list = list()
star_list = list()

for result in search_results:
    #만족도가 없을 수도 있기에 try except문을 활용
    try:
        title_element = result.find_element(By.CSS_SELECTOR, 'span.text__item')
        price_element = result.find_element(By.CSS_SELECTOR, 'div.box__price-seller > strong')
        star_element = result.find_element(By.CSS_SELECTOR, 'span.image__awards-points > span')
        title_list.append(title_element.text)
        price_list.append(price_element.text)
        star_list.append(star_element.text)
    except:
        pass

driver.quit()

In [3]:
def extract_comma(x):
    price = int(x.replace(",", "")) #replace로 쉼표까지 제거하고 int로 변환
    return price

def extract_stars(x):
    ext = re.findall("\d+%", x)
    #데이터가 빈 경우가 있으므로 try, except문 사용
    try:
        stars = int(ext[0].replace('%', "")) #%를 제거한 뒤 int로 변환
    except:
        pass
    return stars if ext else None #데이터가 있을 경우 stars를 반환, 아닐 경우 None을 반환

In [4]:
#3. 적절히 데이터 전처리를 하여 DataFrame 객체로 만들고 Excel 파일로 저장하기 (50점)
#이때, DataFrame의 열은 "상품명, 판매가, 만족도"로 구성되어야 함.
df = pd.DataFrame([title_list, price_list, star_list]).T
df.columns = ['상품명', '판매가', '만족도']

df['판매가'] = df['판매가'].apply(extract_comma)
df['만족도'] = df['만족도'].apply(extract_stars)
df = df.dropna().reset_index().iloc[:, 1:]
df.to_excel('gmarket_soccerball_stars.xlsx')